# Deep dive into the Qumulo API python bindings

**<span style="background-color: #FFBBAA; padding: 5px;">This data is current as of Qumulo Version 2.14.0</span>**

This notebook helps a user explore the Qumulo API python bindings. First, it will list out all of the supported Qunmulo API python modules and functions. **Don't forget to run <code>pip install qumulo_api</code> first.**

After listing all of the defined python functions, it demonstrates one (ore more) functions inside each of python modules. All code examples below are will only read data and configuration from your Qumulo cluster.

In [1]:
import qumulo
import os
import glob
import re
import time
import StringIO
from datetime import datetime
import dateutil.parser as date_parser
from qumulo.rest_client import RestClient

In [2]:
%%javascript
// this will prevent the large output window below from being boxed in.
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
# set your environment variables or fill in the variables below
API_HOSTNAME = os.environ['API_HOSTNAME'] if 'API_HOSTNAME' in os.environ else 'product'
API_USER =     os.environ['API_USER']     if 'API_USER'     in os.environ else 'admin'
API_PASSWORD = os.environ['API_PASSWORD'] if 'API_PASSWORD' in os.environ else ''

### Inspect all of the Qumulo python bindings and show methods

In [4]:
qumulo_lib_path = os.path.dirname(qumulo.__file__) + '/rest'

total_matches = 0
for f in glob.glob(qumulo_lib_path + '/*.py'):
    file_name = os.path.basename(f)
    if file_name == '__init__.py':
        continue
    print ""
    print "-"*80
    print "Area: %s" % (file_name, )
    c = open(f, 'r').read()
    rx_str = '@request.request[ \r\n]+def ([^(]+)\([ \r\n]*conninfo,[ \r\n]*credentials([^\)]*)(.*?)(return|yield)'
    ms = re.findall(rx_str, c, re.S|re.M)
    for m in ms:
        total_matches += 1
        func_name = m[0]
        
        # get arguments
        args = []
        arg_ms = m[1].split(',')
        for arg_m in arg_ms:
            if arg_m.strip() != "":
                args.append(re.sub('=.*', '', arg_m.strip()))

        # method
        method = "GET"
        method_m = re.search('method[ ]*=[ ]*"([A-Z]+)', m[2])
        if method_m is not None:
            method = method_m.group(1)

        # uri, currently more work for fs methods
        uri = "/"
        uri_m = re.search('uri[ ]*=.*?"([^"]+)', m[2])
        if uri_m is not None:
            uri = uri_m.group(1)
        uri_m = re.search('uri[ ]*=.*?\'([^\']+)', m[2])
        if uri_m is not None:
            uri = uri_m.group(1)

        print "   rc.%s.%s(%s)" % (file_name.replace('.py', ''), 
                                   func_name, 
                                   ', '.join(args[:4]) + (' ...' if len(args)>4 else ''))



--------------------------------------------------------------------------------
Area: auth.py
   rc.auth.change_password(old_password, new_password)
   rc.auth.who_am_i()
   rc.auth.auth_id_to_all_related_identities(auth_id)
   rc.auth.posix_uid_to_all_related_identities(posix_uid)
   rc.auth.posix_gid_to_all_related_identities(posix_gid)
   rc.auth.sid_to_all_related_identities(sid)
   rc.auth.local_username_to_all_related_identities(username)
   rc.auth.get_identity_attributes(auth_id)
   rc.auth.set_identity_attributes(auth_id, attributes)
   rc.auth.delete_identity_attributes(auth_id)
   rc.auth.user_defined_mappings_set(mappings)
   rc.auth.user_defined_mappings_get()
   rc.auth.clear_cache()
   rc.auth.expand_identity(_id, aux_equivalent_ids, aux_group_ids)

--------------------------------------------------------------------------------
Area: ldap.py
   rc.ldap.settings_set_v2(bind_uri, base_distinguished_names, use_ldap, user ...)
   rc.ldap.settings_get_v2()
   rc.ldap.setti

### Create a new python REST client instance and login.

In [5]:
# Create a new reset client and login.
rc = RestClient(API_HOSTNAME, 8000)
rc.login(API_USER, API_PASSWORD)

### Show Qumulo API python client library version and Qumulo cluster software version

In [6]:
# if qumulo_api is installed via pip, this will return its version.
cmd_output = !pip show qumulo_api

# parse out results if "pip show"
pip_version = '! Unknown !'
for line in cmd_output:
    parts = line.split(':')
    if parts[0] == 'Version':
        pip_version = parts[1]

print("Qumulo API python library version: %s" % (pip_version.strip(),))

# get the Qumulo cluster software version via the API
cluster_version = rc.version.version()
print("Qumulo Cluster software version: %s" % (cluster_version['revision_id'].replace('Qumulo Core ', ''),))

# How old is the current build on the Qumulo cluster?
build_time = int(date_parser.parse(cluster_version['build_date']).strftime('%s'))
cur_time = time.time()
print("Qumulo Cluster software version is: %d days old." % ((cur_time - build_time)/(60*60*24),))

Qumulo API python library version: 2.14.0
Qumulo Cluster software version: 2.14.0
Qumulo Cluster software version is: 31 days old.


## Active Directory (ad) - list AD status

In [7]:
# Show the current status of the Cluster's AD relationship
rc.ad.list_ad()

{u'base_dn': u'CN=Users,DC=ad,DC=qumulo,DC=com',
 u'dcs': [{u'address': u'10.220.248.156', u'name': u'cfwdc1.ad.qumulo.com'}],
 u'domain': u'ad.qumulo.com',
 u'domain_netbios': u'AD',
 u'ldap_connection_states': [{u'base_dn_vec': [u'CN=Users,DC=ad,DC=qumulo,DC=com'],
   u'bind_account': u'product$',
   u'bind_domain': u'ad.qumulo.com',
   u'node_id': 1,
   u'servers': [{u'bind_uri': u'ldap://cfwdc1.ad.qumulo.com',
     u'kdc_address': u'10.220.248.156'}]},
  {u'base_dn_vec': [u'CN=Users,DC=ad,DC=qumulo,DC=com'],
   u'bind_account': u'product$',
   u'bind_domain': u'ad.qumulo.com',
   u'node_id': 2,
   u'servers': [{u'bind_uri': u'ldap://cfwdc1.ad.qumulo.com',
     u'kdc_address': u'10.220.248.156'}]},
  {u'base_dn_vec': [u'CN=Users,DC=ad,DC=qumulo,DC=com'],
   u'bind_account': u'product$',
   u'bind_domain': u'ad.qumulo.com',
   u'node_id': 3,
   u'servers': [{u'bind_uri': u'ldap://cfwdc1.ad.qumulo.com',
     u'kdc_address': u'10.220.248.156'}]},
  {u'base_dn_vec': [u'CN=Users,DC=ad,DC

## analytics - timeseries data

In [8]:
# Get the latest minute's metrics from the timeseries data endpoint.
# This data is used on the Qumulo web application's dashboard home page.
# Show the average value for the last minute for each series.
data = rc.analytics.time_series_get(begin_time=int(time.time() - 60))
for series in data:
    # skip totals since they are duplicated by the other metrics
    if 'total' in series['id']:
        continue
    print "%22s - %11s" % (series['id'],
                       round(sum(series['values']) / len(series['values']), 1))

        iops.read.rate -         9.9
       iops.write.rate -         8.7
 reclaim.deferred.rate -         0.0
 reclaim.snapshot.rate -         0.0
  throughput.read.rate -   4119905.7
 throughput.write.rate -   1960931.9


## Authentication and Authorization (auth) - get related identities

In [9]:
for a in rc.auth.local_username_to_all_related_identities('admin'):
    print "%(id_type)s - %(id_value)s" % a

NFS_UID - 0
LOCAL_GROUP - Users
LOCAL_USER - admin


## cluster - list nodes

In [10]:
for n in rc.cluster.list_nodes():
    print "%(node_name)s/%(id)s - %(model_number)s" % n

product-1/1 - Q0626
product-2/2 - Q0626
product-3/3 - Q0626
product-4/4 - Q0626


## dns - resolve names from ip addresses

In [11]:
for d in rc.dns.resolve(['127.0.0.1', '10.20.217.62', '192.168.0.1', '192.168.1.1', 
                       '192.168.154.1', '172.16.1.1', '10.120.246.43', '10.10.1.1']):
    print("%(ip_address)15s - %(result)10s - %(hostname)s" % d)

      127.0.0.1 -         OK - localhost
   10.20.217.62 -         OK - dyn-010-020-217-062.corp.qumulo.com
    192.168.0.1 -  NOT_FOUND - 
    192.168.1.1 -  NOT_FOUND - 
  192.168.154.1 -  NOT_FOUND - 
     172.16.1.1 -  NOT_FOUND - 
  10.120.246.43 -         OK - af9-100g.eng.qumulo.com
      10.10.1.1 -  NOT_FOUND - 


## File System (fs) - list files and read a file

In [12]:
path = '/'
dir_ent = rc.fs.read_directory(path=path)
for d in dir_ent['files']:
    if d['type'] == 'FS_FILE_TYPE_FILE':
        fw = StringIO.StringIO()
        print("Read file %(name)s which is %(size)s bytes, and print first 80 bytes" % d)
        rc.fs.read_file(fw, path = path + d['name'])
        print fw.getvalue()[:80]
        break


Read file ._.DS_Store which is 4096 bytes, and print first 80 bytes
    Mac OS X            	   2  �     �          @                    


## groups - list all groups in Qumulo

In [13]:
for g in rc.groups.list_groups():
    print "%(gid)6s %(id)6s %(name)16s %(sid)50s" % g


 65534    514           Guests       S-1-5-21-794736294-2450901474-4127211561-514
          513            Users       S-1-5-21-794736294-2450901474-4127211561-513


## network - show current connection counts to all nodes

In [14]:
for c in rc.network.connections():
    print "Node %2s connection count: %4s  ----  First %s: %s" % (
            c['id'], 
            len(c['connections']),
            min(len(c['connections']), 10), 
            ', '.join([d['network_address'] + '/' + d['type'].replace('CONNECTION_TYPE_', '') 
                       for d in c['connections']]))

Node  1 connection count:    2  ----  First 2: 10.20.218.226/NFS, 10.220.246.241/NFS
Node  2 connection count:    0  ----  First 0: 
Node  3 connection count:    0  ----  First 0: 
Node  4 connection count:    1  ----  First 1: 10.220.246.245/NFS


## nfs - list shares

In [15]:
for share in rc.nfs.nfs_list_shares():
    print "%(export_path)s -> %(fs_path)s - %(description)s" % share

/ -> / - 
/test-export -> /test-export - 
/testing-tommy -> /testing-tommy - 
/Zaher  -> /Zaher  - 
/qumulo-audit-logs -> /qumulo-audit-logs - 


## node_state - get current node state

In [16]:
print rc.node_state.get_node_state()
# (only prints the state of the node the rest client is currently connected to)

{u'state': u'ACTIVE', u'node_id': 1, u'cluster_id': u'fa3b2fa0-57bc-4e0b-9d75-5f5ddff5a34c'}


## quota - list first 20 quotas and capacity used for each

In [20]:
quota_count = 0
for qd in rc.quota.get_all_quotas_with_status():
    for q in qd['quotas']:
        quota_count += 1
        if quota_count > 20:
            break
        print "%(path)s - id: %(id)s - %(capacity_usage)s bytes used of %(limit)s" % q

/test/ - id: 26000004 - 1478684233728 bytes used of 10000000000000
/users/dept/u-400/ - id: 14243563876 - 4096 bytes used of 1000000000
/users/dept/u-3000/ - id: 14243563877 - 4096 bytes used of 1000000000
/users/dept/u-401/ - id: 14243563879 - 4096 bytes used of 1000000000
/users/dept/u-3001/ - id: 14243563880 - 4096 bytes used of 1000000000
/users/dept/u-402/ - id: 14243563881 - 4096 bytes used of 1000000000
/users/dept/u-3002/ - id: 14243563882 - 4096 bytes used of 1000000000
/users/dept/u-403/ - id: 14243563883 - 4096 bytes used of 1000000000
/users/dept/u-3003/ - id: 14243563884 - 4096 bytes used of 1000000000
/users/dept/u-404/ - id: 14243563885 - 4096 bytes used of 1000000000
/users/dept/u-3004/ - id: 14243563886 - 4096 bytes used of 1000000000
/users/dept/u-405/ - id: 14243563887 - 4096 bytes used of 1000000000
/users/dept/u-3005/ - id: 14243563888 - 4096 bytes used of 1000000000
/users/dept/u-406/ - id: 14243563889 - 4096 bytes used of 1000000000
/users/dept/u-3006/ - id: 1424

## smb - list shares

In [21]:
for share in rc.smb.smb_list_shares():
    print "%(share_name)s -> %(fs_path)s - %(description)s" % share

Files -> / - 
jsturgeon -> /home/jsturgeon - 
smb-share -> /smb-share - 
ozgur -> /home/ozgur - just demo
stefan1 -> /home/stefan1 - for testing audit - read only 
qumulo-audit-logs -> /qumulo-audit-logs - 
LasVegas -> /LasVegas - BRIAN: Ultimate Software POC
no-access -> /no-access - 
test -> /test - testing test test


## snapshots - list directory-level snapshots

In [25]:
for snap in rc.snapshot.list_snapshots()['entries'][:10]:
    print("%(name)s - %(source_file_id)s - %(directory_name)s - %(timestamp)s" % snap)

Replication from baby-groot - 14179051126 - 396201_Replication from baby-groot - 2019-08-30T18:25:51.016772228Z
Replication from baby-groot - 14179051126 - 425374_Replication from baby-groot - 2019-09-27T13:21:54.835507389Z
replication_to_music - 153000050 - 438360_replication_to_music - 2019-10-09T00:01:29.191201784Z
replication_from_newgravytrain - 14152071783 - 454346_replication_from_newgravytrain - 2019-10-24T17:56:05.6006577Z
Cluster Root - 2 - 454350_Cluster Root - 2019-10-27T16:09:00.000528751Z
Cluster Root - 2 - 455937_Cluster Root - 2019-11-03T17:09:00.00080422Z
user u-0 - 14243793681 - 455938_user u-0 - 2019-11-08T16:42:54.798627239Z
user u-0 - 14243793681 - 455939_user u-0 - 2019-11-08T16:43:30.746198256Z
user u-1 - 14243793683 - 455940_user u-1 - 2019-11-08T16:43:30.754319848Z
user u-2 - 14243793685 - 455941_user u-2 - 2019-11-08T16:43:30.759718014Z


## support - check support/monitoring status

In [26]:
rc.support.get_config()

{u'enabled': True,
 u'mq_host': u'missionq.qumulo.com',
 u'mq_port': 443,
 u'mq_proxy_host': u'',
 u'mq_proxy_port': 0,
 u'period': 60,
 u's3_proxy_disable_https': False,
 u's3_proxy_host': u'monitor.qumulo.com',
 u's3_proxy_port': 443,
 u'vpn_enabled': True,
 u'vpn_host': u'ep1.qumulo.com'}

## time_config - get current time status

In [27]:
rc.time_config.get_time_status()

{u'config': {u'ntp_servers': [u'0.qumulo.pool.ntp.org',
   u'1.qumulo.pool.ntp.org'],
  u'use_ad_for_primary': True},
 u'time': u'2019-11-20T02:22:33.248152991Z'}

## users - list users

In [28]:
for user in rc.users.list_users():
    print("%(name)10s - %(uid)5s - %(id)5s - %(primary_group)4s - %(sid)s" % user)

     brian -       -  1009 -  513 - S-1-5-21-794736294-2450901474-4127211561-1009
    stefan -       -  1008 -  513 - S-1-5-21-794736294-2450901474-4127211561-1008
  ftp-user -       -  1006 -  513 - S-1-5-21-794736294-2450901474-4127211561-1006
     guest - 65534 -   501 -  514 - S-1-5-21-794736294-2450901474-4127211561-501
     admin -     0 -   500 -  513 - S-1-5-21-794736294-2450901474-4127211561-500
     jason -       -  1000 -  513 - S-1-5-21-794736294-2450901474-4127211561-1000
     tommy -       -  1010 -  513 - S-1-5-21-794736294-2450901474-4127211561-1010
  api-user -       -  1007 -  513 - S-1-5-21-794736294-2450901474-4127211561-1007
